#### В державних органах можуть працювати багато родичів, що може ставити перепони для інших кандидатів на посаду. Тому метою цього аналізу є перевірка цієї гіпотези. Дані для аналізу були взяті з декларацій державних службовців - [declarations.com.ua](https://declarations.com.ua/static/data/full_export.json.bz2). Об'єдниний архів 2.5 ГБ, в розархівованоми вигляді 82 ГБ. Перед тим як перейти до аналізу з декларації за 2018 була відібрана інформація: фамілія, ім'я, по батькові, місце роботи та посада та збережена в окремому файлі.

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
data = pd.read_csv('infocards.csv')
'В сирих даних {} записів, де є {} неунікальних організацій'.format(data.shape[0], data.office.nunique())

'В сирих даних 1268732 записів, де є 342335 неунікальних організацій'

#### Переводимо текстову інформацію переведена у верхній регістр. 

In [3]:
data.loc[:,'first_name'] = data.first_name.str.upper()
data.loc[:,'patronymic'] = data.patronymic.str.upper()
data.loc[:,'last_name'] = data.last_name.str.upper()
data.loc[:,'office'] = data.office.str.upper()

#### Видаляємо абревіатури в назві місця роботи та видаляємо лапки, що залишились.

In [4]:
abbs = ['ГУ', 'ВП', 'АТ', 'ПАТ', 'ТОВ', 'ДП', 'КНП', 'КП', 'ГРОМАДСЬКА СПІЛКА', 'ДЕРЖАВНА УСТАНОВА', '"']
for abb in abbs:
    data.loc[:,'office'] = data.office.str.replace(abb + '\s*', '')
'{} записів, де є {} організацій'.format(data.shape[0], data.office.nunique())

'1268732 записів, де є 292490 організацій'

#### Після цього в ручному режимі були перевірені місця роботи, що вказали декларанти. Видаляємо нерелевантні.

In [5]:
# Delete frequent non-relevant offices
NROFFS = ['ПЕНСІОНЕР', 'ПЕНСІОНЕРКА', 'ПРАЦЮЮ', 'ПРАЦЮЄ', 'ДОМОГОСПОДАРКА',
          'ФІЗИЧНА ОСОБА - ПІДПРИЄМЕЦЬ', 'ФОП', 'ПРИВАТНИЙ ПІДПРИЄМЕЦЬ', 'ФІЗИЧНА ОСОБА-ПІДПРИЄМЕЦЬ', 
          'ФІЗИЧНА ОСОБА ПІДПРИЄМЕЦЬ', 'ПІДПРИЄМЕЦЬ',
          'БЕЗРОБІТНИЙ', 'БЕЗРОБІТНІЙ', 'БЕЗРОБІТНЯ', 'БЕЗРОБІТНА', 'ТИМЧАСОВО НЕ ПРАЦЮЮ', 
          'ТИМЧАСОВО НЕ ПРАЦЮЄ', 'НЕ ПРАЦЮЮ', 'НЕ ПРАЦЮЄ',
          'ВІЙСЬКОВОСЛУЖБОВЕЦЬ ЗБРОЙНИХ СИЛ УКРАЇНИ', 'ВІЙСЬКОВА ЧАСТИНА', 'ВІЙСЬКОВОСЛУЖБОВЕЦЬ', 
          'ЗСУ', 'ВІЙСЬКОВА ЧАСТИНА',
          'УКРАЇНИ', 'ДЕПУТАТ СІЛЬСЬКОЇ РАДИ', 'ДЕПУТАТ', 'СІЛЬСЬКА РАДА', 'МІСЦЕВА ПРОКУРАТУРА', 
          'В ОДЕСЬКІЙ ОБЛАСТІ', 'М.КИЇВ', 'ДЕТЕКТИВ', 'НЕМАЄ']
data = data[~data.office.isin(NROFFS)]
'{} записів, де є {} організацій'.format(data.shape[0], data.office.nunique())

'1250822 записів, де є 292470 організацій'

In [6]:
offices = list(map(lambda x: x.strip().upper(), open('offices.txt').readlines()))
for off in offices:
    data.loc[data.office.str.contains(off), 'office'] = off
'{} записів, де є {} організацій'.format(data.shape[0], data.office.nunique())

'1250822 записів, де є 290012 організацій'

#### Прийшла черга видалити дублікати, бо деякі службовці можуть подати кілька декларацій за рік.

In [7]:
# Drop duplicated declarations
data = data.drop_duplicates(['first_name', 'patronymic', 'last_name', 'office'])
'{} записів, де є {} організацій'.format(data.shape[0], data.office.nunique())

'1010521 записів, де є 290012 організацій'

#### Видалямо офіси, "працюють" меньше 10 осіб.

In [8]:
offices = data.office.value_counts()
offices = offices[offices > 10]
data = data[data.office.isin(offices.index)]
'{} записів, де є {} організацій'.format(data.shape[0], data.office.nunique())

'567660 записів, де є 14429 організацій'

#### Додаємо стовпчик з коефіцієнтом частоти фамілії (100 * кількість людей з такою ж фамілією / на кількість всіх)

In [9]:
surnames = data.groupby('last_name').agg({'id': 'count'}).reset_index().rename(columns={'id': 'last_name_k'})
surnames = surnames.assign(last_name_k=lambda x: 100 * x.last_name_k / data.shape[0])
data = data.merge(surnames, on='last_name', how='left')

#### Рахуємо кількість однакових фамілій в одній структурі

In [10]:
grouped = data.groupby(['office', 'last_name'])\
    .agg(surname_count=('id', 'count'), surname_coef=('last_name_k', 'first')).reset_index()

#### Рахуємо кількість працівників в одній структурі

In [11]:
employees = grouped.groupby('office').agg(employees=('surname_count', 'sum'))
grouped = grouped.merge(employees, on='office', how='left')

#### Викидаємо фамілії, які зустрічаються лише раз

In [12]:
grouped = grouped[grouped.surname_count > 1]

#### Вираховуємо ранг для кожної фамілії (килькість / коєфіцієнт)

In [13]:
grouped = grouped.assign(rank=lambda x: x.surname_count / x.surname_coef)

#### Зберігаємо фаміліїї та їх кількість в окремій змінній та видаляємо непотрібні стовпчики

In [14]:
grouped['surnames'] = grouped.last_name + '-' + grouped.surname_count.astype(str)
grouped = grouped.drop(['surname_coef', 'last_name', 'surname_count'], axis=1)

#### Рахуємо суммарний ранг для кожної структури та ділимо на кількість робітників

In [15]:
grouped = grouped.groupby('office')\
    .agg(rank=('rank', 'sum'), employees=('employees', 'first'), surnames=('surnames', list)).reset_index()
grouped['rank'] = grouped['rank'] / grouped['employees']

##### Дивимось на структури де більше 500 співробітників та більше всього однофамільців

In [17]:
grouped[grouped.employees > 500].sort_values('rank', ascending=False).head(20)

,office,rank,employees,surnames
2367,ЗАКАРПСЬКА МИТНИЦЯ ДФС,122.405830,857,"[АНДРІЙЧУК-2, БАБИЧ-2, БАРАН-2, БОДНАР-2, ВАСИ..."
1141,ВОЛИНСЬКА МИТНИЦЯ ДФС,106.058402,545,"[ЄВТУШОК-3, АНТОНЮК-2, БІСЮК-3, БОНДАР-2, БОНД..."
3039,ЛЬВІВСЬКА МИТНИЦЯ ДФС,89.740447,845,"[ІВАНУСА-3, БІДНИК-2, БІЛИК-2, БАНАХ-2, БАРАН-..."
1502,ГОЛОВНЕ УПРАВЛІННЯ ПЕНСІЙНОГО ФОНДУ УКРАЇНИ В ...,72.847419,712,"[АНТОНЮК-2, АНТОШИНА-2, БЕРНАЦЬКА-2, БОЙКО-2, ..."
3411,НВКГ ЗОРЯ-МАШПРОЕКТ,71.793333,705,"[ІВАНОВ-4, ІСАКОВ-2, АЛЕКСЄЄНКО-2, АРТАМОНОВ-2..."
1511,ГОЛОВНЕ УПРАВЛІННЯ ПЕНСІЙНОГО ФОНДУ УКРАЇНИ У ...,67.390799,1148,"[ІВАНЕЦЬ-2, ІВАНЮК-3, ІВАХОВИЧ-2, БІЛЕЦЬКА-2, ..."
1469,ГОЛОВНЕ УПРАВЛІННЯ НАЦІОНАЛЬНОЇ ПОЛІЦІЇ В ЧЕРН...,66.833189,835,"[ЄВТУШЕНКО-2, ЄРМОЛЕНКО-3, ЄФИМЕНКО-2, ІВАНОВ-..."
1779,ДЕРЖАВНА ІНСПЕКЦІЯ ЕНЕРГЕТИЧНОГО НАГЛЯДУ УКРАЇНИ,60.091352,533,"[ІВАНОВ-2, БАБИЧ-2, БОЖКО-2, БОНДАР-3, БОНДАРЕ..."
5269,ХАРКІВСЬКА МИТНИЦЯ ДФС,59.843834,517,"[БЄЛЄВЦОВ-2, БІЛЮК-2, БАШТЕНКО-2, БОНДАРЕНКО-2..."
3963,ПРОКУРУРА ХАРКІВСЬКОЇ ОБЛАСТІ,57.915419,507,"[ЄВТУШЕНКО-2, ЄНА-2, АНУШКЕВИЧ-2, БОЖКО-2, БОН..."
